In [ ]:
# !! pip install fastai

In [ ]:
# ones = u.read_ones(data_path / 'train.csv')
# u.add_more_time(ones)
# ones = ones.rename(columns={'sid': 'segment_id', 'day': 'weekday'})

# vehinj = u.read_vehinj(Path('../data/SANRAL_v2'))

# field_combs = [
#     tuple([('segment_id')]), tuple(['hour']), tuple(['weekday']), tuple(['month']),
#     ('segment_id', 'hour'), ('segment_id', 'weekday'), ('segment_id', 'month'), ('weekday', 'month'), ('hour', 'weekday'),
#     ('segment_id', 'weekday', 'hour'), ('segment_id', 'weekday', 'month')
# ]


# train = u.select_by_time(train_full, '2017-01-01', '2018-10-01')
# u.add_statistic(data=train, stat_data=ones,
#                 field_combs=field_combs, prefix='hist_',
#                 tstart='2016-01-01', tend='2017-01-01',
#                )

# val = u.select_by_time(train_full, '2018-10-01', '2019-01-01')
# u.add_statistic(data=val, stat_data=ones,
#                 field_combs=field_combs, prefix='hist_',
#                 tstart='2016-01-01', tend='2018-10-01',
#                )

# u.add_statistic(data=test, stat_data=ones,
#                 field_combs=field_combs, prefix='hist_',
#                 tstart='2016-01-01', tend='2019-01-01',
#                )

# x_cols = cols
# x_cols.extend([c for c in train.columns if c.startswith('hist_')])

# train = u.select_by_time(train_full, '2016-01-01', val_tstart, time_col='datetime')
# val = u.select_by_time(train_full, val_tstart, '2019-01-01', time_col='datetime')

# def prepare(df):
#     df[cat_cols] = df[cat_cols].replace(np.nan, 'NAN').astype('object')
#     df[cat_cols] = df[cat_cols].astype('str')
    

# prepare(train_full)
# prepare(test)

# val = train_full.loc[ids_val]
# val.reset_index(inplace=True)

# u.run_many_th_experiment(val, np.array(predictions), th_score=0.12)

# train_full = train_full[train_full.datetime < pd.Timestamp('2018-04-01')]  # Only for fast debug
# train_full.reset_index(inplace=True)


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from fastai.tabular import *
from fastai.callbacks.tracker import EarlyStoppingCallback, SaveModelCallback
from pathlib import Path
import src.data_utils as u
import src.fai_utils as fu
from tqdm.auto import tqdm
from collections import Counter
from sklearn.metrics import f1_score, precision_score, recall_score

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 100)

u.random_seed(42)

data_path = Path('../data')

In [ ]:
train_full, test, all_cols, cont_cols, cat_cols = u.read_data(data_path)

# Fast ai

In [ ]:
for f in cont_cols:
    test[f] = test[f].fillna(0)
    train_full[f] = train_full[f].fillna(0)

In [ ]:
def get_folds(train):
    # months = ['01', '03', '06', '09', '12']
    months = ['01', '03']
    
    folds = []
    for i_fold in range(len(months) - 1):
        tstart = pd.Timestamp('2018-' + months[i_fold] + '-01')
        tend = pd.Timestamp('2018-' + months[i_fold + 1] + '-01')

        tt = train.datetime
        ids_val = ((tstart <=  tt) & (tt < tend)).to_numpy().nonzero()[0]
        folds.append(ids_val)
    
    return folds
        
    
folds = get_folds(train_full)

In [ ]:
probas_arr, ths, val_scores = fu.fit_predict_cv(
    train=train_full, test=test, val_ids=folds,
    cat_cols=cat_cols, cont_cols=cont_cols, draw_plot=False
)

In [ ]:
0.06964964119881807
0.08447399296050058 no weights

In [ ]:
probas = predictions_arr.mean(axis=0)

In [ ]:
u.estimate(learn, validation_pool=None, y_true=train_full.y[ids_val],
           th_start=0, th_stop=1, steps=20)